# Load data

In [1]:
import os
import gc
import pandas as pd


from _00_data_loader import get_data_split_author


import pickle
from collections import Counter
from tqdm import tqdm
import numpy as np
import random

import json
import ast
from sklearn.metrics import *

import matplotlib.pyplot as plt
import seaborn as sns

from _00_prompts import FIND_SUPPORT_PROMPT, FIND_SUPPORT_PROMPT_WITHVALUES, FIND_FLIPPED_PROMPT_WITHVALUES

In [31]:
all_authors = set(df['author'].tolist())

In [38]:
for author in tqdm(all_authors):
    author_subIDs_all = []
    try:
        train_data = np.load(f"{os.getcwd()}/data/byRedditor/{author}_train_0.npy",  allow_pickle=True)
    except:
        continue
    valid_data = np.load(f"{os.getcwd()}/data/byRedditor/{author}_valid_0.npy",  allow_pickle=True)
    test_data = np.load(f"{os.getcwd()}/data/byRedditor/{author}_test.npy",  allow_pickle=True)

    author_subIDs_all += [elem['subID'] for elem in train_data]
    author_subIDs_all += [elem['subID'] for elem in valid_data]
    author_subIDs_all += [elem['subID'] for elem in test_data]

    for _subID in author_subIDs_all:
        indices = [elem[0] for elem in ID_to_closest_bySituation[f'{_subID}###{author}'][:5]]
        subIDs = [situIndex_to_subID[elem] for elem in indices]
        close_schwartz, close_value, close_abst_value, close_abst_value_clustered = [],[],[],[]
        for close_subID in subIDs:
            close_schwartz.append(author_to_subID_to_inst[author][close_subID]['schwartz'])
            close_value.append(author_to_subID_to_inst[author][close_subID]['value'])
            close_abst_value.append(author_to_subID_to_inst[author][close_subID]['abst_value'])
            close_abst_value_clustered.append(author_to_subID_to_inst[author][close_subID]['abst_value_clustered'])
        author_to_subID_to_inst[author][_subID]['close_schwartz'] = [str(elem) for elem in close_schwartz]
        author_to_subID_to_inst[author][_subID]['close_value'] = [str(elem) for elem in close_value]
        author_to_subID_to_inst[author][_subID]['close_abst_value'] = [str(elem) for elem in close_abst_value]
        author_to_subID_to_inst[author][_subID]['close_abst_value_clustered'] = [str(elem) for elem in close_abst_value_clustered]
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:05<00:00, 43.41it/s]


In [39]:
with open(f"{os.getcwd()}/data/author_to_subID_to_inst_withClosest.pkl", "wb") as f:
    pickle.dump(author_to_subID_to_inst, f)
    # author_to_subID_to_inst = pickle.load(f)

In [15]:
_data = np.load(f"{os.getcwd()}/data/byRedditor/6poundpuppy_test.npy",  allow_pickle=True)

In [28]:
_data[0]['subID']

'n6tl5w'

In [18]:
with open(f"{os.getcwd()}/data/ID_to_closest_bySituation.pkl", "rb") as f:
    ID_to_closest_bySituation = pickle.load(f)

In [25]:
with open(f'{os.getcwd()}/embedding/Situation_text-embedding-3-large.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)

situIndex_to_subID = {}
for idx, elem in enumerate(situ_dist_data['ID']):
    situIndex_to_subID[idx] = elem
del situ_dist_data

In [26]:
situIndex_to_subID[8673]

'mvc4i0'

In [23]:
with open(f"{os.getcwd()}/data/author_to_controversialIndex_byRedditor.pkl", "rb") as f:
    author_to_controversialIndex_byRedditor = pickle.load(f)

In [21]:
ID_to_closest_bySituation['n6tl5w###6poundpuppy']

[(8673, np.float64(0.8795769271699445)),
 (7844, np.float64(0.8807378543880104)),
 (14669, np.float64(0.9040137597477955)),
 (10293, np.float64(0.9079063253095063)),
 (8732, np.float64(0.9104272944478615)),
 (14549, np.float64(0.9256268506815878)),
 (7364, np.float64(0.9334687381242012)),
 (5594, np.float64(0.9401882981453109)),
 (16026, np.float64(0.9490223653669706)),
 (6367, np.float64(0.9510273114700928)),
 (1748, np.float64(0.9529758088684146)),
 (9182, np.float64(0.9618555147287973)),
 (1510, np.float64(0.9626269819924175)),
 (4114, np.float64(0.9636477644894135)),
 (259, np.float64(0.9655107478553765)),
 (5414, np.float64(0.9677117797615442)),
 (9327, np.float64(0.9680301889425461)),
 (12940, np.float64(0.9796561747602333)),
 (15251, np.float64(0.9798836397305181)),
 (154, np.float64(0.9824694782112241)),
 (15491, np.float64(0.9886006713704026)),
 (13613, np.float64(0.9916647567211748)),
 (3014, np.float64(0.9934866718777322)),
 (14026, np.float64(0.9940368723454217)),
 (13973, 

In [22]:
with open(f"{os.getcwd()}/data/author_to_subID_to_inst.pkl", "rb") as f:
    author_to_subID_to_inst = pickle.load(f)

In [27]:
author_to_subID_to_inst['6poundpuppy']['mvc4i0']

{'title': 'telling the teacher to stop making comments on my wheelchair?',
 'selftext': 'I\'m 18f, have a pretty messed up pair of legs, have had since birth, I can walk but am an ambulatory wheelchair user. I\'m currently due an upgrade for my chair, I\'ve had it close to 7 years and it\'s a bit messed up. It\'s gotten pretty uncomfortable and it makes noise but like I said, I\'ve had it 7 years and I\'ve grown rather attatched to it. We\'re saving to pay for the new one at the moment.\n\nI have one teacher, my English teacher, who constantly makes comments about how banged up looking it is, and gets pretty pissed any time I dare move and it makes noise. She says it\'s distracting. The comments about the appearance of the chair annoy me a lot because it\'s hardly going to look brand new after 7 years of constant use.\n\nShe made a comment this morning along the lines of "You know, you should really get a new one, that one looks like it\'s about to collapse under you". I got really mad

In [17]:
_data[0]

{'subID': 'n6tl5w',
 'title': 'refusing to wish my friend happy teacher appreciation week for simply helping her kid with virtual learning when I’m an actual teacher?',
 'selftext': 'I [31f] am a middle school ELA teacher.  I have a friend, also [31f], who is not a teacher.\n\nShe posted on her Facebook page the following message: “Happy appreciation week to the real teachers out there: parents!  Kids couldn’t do this virtual learning without us!” followed by a lot of hashtags about being mom strong and what not.\n\nI didn’t comment on FB because I’m 31 and don’t do FB arguments.  I did, however, bring it up in a conversation we were having over the phone.  We were talking about her child’s virtual schooling (he’s 8) when she said how hard teaching is.  She says this a lot, but I have always blown it off because I never thought she actually believed she was a teacher until this moment.  That’s when I asked her if she really believed that status update she posted, and she said absolutel

In [2]:
df = pd.read_csv(f'{os.getcwd()}/data/data_active_authorExt_withAbstValues.tsv', sep='\t')
df.head(2)

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats,id
0,k8zq74,not doing my stepchildren's hair?,"Hello, I'm 30F married to my partner 51M and h...",NoApollonia,The children are both old enough to choose if ...,1,"[""BM's value of appearance and expectation for...","```json\n{\n ""Conflicting Values"": [\n ...",Social Appearance and Judgment vs. Family Dyna...,Societal Norms and Individual Impact vs. Perso...,Self-Expression and Comfort > Parenting Author...,Themes of Identity Exploration > Parenting Dec...,k8zq74###NoApollonia
1,k93tjn,insulting my wife in front of our son.,Growing up I had faced a few key issues that I...,NoApollonia,Don't disrespect your wife in front of your so...,0,['Personal Achievement vs. Academic Excellence...,"```json\n{\n ""Conflicting Values"": [\n ...",Personal Career Satisfaction vs. Academic Pres...,Career Development and Satisfaction Themes vs....,Academic Pressure and Expectations > Personal ...,Educational Themes and Responsibilities > Care...,k93tjn###NoApollonia


In [3]:
with open(f"{os.getcwd()}/data/author_to_subID_to_inst.pkl", "rb") as f:
    author_to_subID_to_inst = pickle.load(f)

with open(f"{os.getcwd()}/data/ID_to_closest_bySituation.pkl", "rb") as f:
    ID_to_closest = pickle.load(f)

with open(f'{os.getcwd()}/embedding/Situation_text-embedding-3-large.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
situIndex_to_subID = {}
for idx, elem in enumerate(situ_dist_data['ID']):
    situIndex_to_subID[idx] = elem
del situ_dist_data

In [4]:
with open(f"{os.getcwd()}/data/author_to_controversialIndex_byRedditor.pkl", "rb") as f:
    author_to_testHardIdx = pickle.load(f)

author_cnt = {_author:len(author_to_testHardIdx[_author]) for _author in author_to_testHardIdx}
author_cnt = sorted(author_cnt.items(), key=lambda x:x[1], reverse=True)
AUTHORS = [elem[0] for elem in author_cnt[:100]]

In [5]:
_author = AUTHORS[0]
_train_set, _valid_set, _test_set = get_data_split_author("/data/byRedditor", _author, 100000, 100000, 100000, return_dict=False)

In [6]:
indices = [i for i, x in enumerate(author_to_testHardIdx[_author]) if x == 1]
print(indices[:10])

[8, 22, 29, 33, 36, 59, 60, 77, 78, 107]


In [7]:
_test_set[33]

{'subID': 'ey9ou3',
 'title': 'getting mad at my boyfriend for not picking me up at the airport when i flew over to visit him?',
 'selftext': 'I’m currently working out of state for about 6 months. Last weekend I decided to visit him for his birthday and flew back about 1000 miles to see him. I took a few days off for this.\n\nWhen I arrive it’s thunderstorming and dark, his house is 30 minutes from the airport. I called to ask him if he was on the way to pick me up but he said no he was at home and he didnt feel safe driving in this weather. To be fair to him he does have vision problems and has a lot of trouble seeing in the dark even with corrective lenses. Instead he said that he’s calling me a lyft.\n\nLyft arrives in a few minutes and arrives at his house half an hour later on. I’m clearly upset and he asks why and I tell him that he wouldn’t even pick me up and just plays things too safe. He gets upset at me and says he really doesnt want to drive 30 minutes when he cant see and

In [8]:
_ID = _test_set[33]['subID'] + '###' + _author
_closest_subIDs = [situIndex_to_subID[elem[0]] for elem in ID_to_closest[_ID]]
_closest_subIDs = [elem for elem in _closest_subIDs if elem != _test_set[33]['subID']]

In [9]:
judgment_to_word = {1: 'Acceptable', 0: 'Unacceptable'}
_fewshot_situations = "###\n\n"
for _subID in _closest_subIDs[:5]:
    _inst = author_to_subID_to_inst[_author][_subID]

    _fewshot_situations += f"[Situation] {_inst['title']}\n"
    _fewshot_situations += f"[Person X's comment] {_inst['comment']}\n"
    _fewshot_situations += f"[Person X's value trade-off] {_inst['abst_value']}\n"
    _fewshot_situations += f"[Person X's judgment] {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
    _fewshot_situations += "\n###\n\n"

In [10]:
print(_fewshot_situations)

###

[Situation] just showing up at his home?
[Person X's comment] He answered for your roommate but not for you so he was avoiding you. You are also not important to him or he would have prioritized texting and calling you, even if he was busy. He likely has another woman or has just checked out of the relationship. Stop tolerating this behavior by staying with him. Break up with him and never allow another man to treat you like this and make you feel like the crazy person when you are not.
[Person X's value trade-off] Lack of Connection > Intent and Misinterpretation
Communication Expectations > Interpersonal Dynamics and Frustrations
[Person X's judgment] 1 (Acceptable)

###

[Situation] for making my boyfriend pay for my $35 Uber to work?
[Person X's comment] he should not have taken muscle relaxers knowing that he had agreed to drive you. And after doing that he shouldn’t have offered to pay for your Uber without you asking. To me it sounds like he wanted your company and lied to 

In [11]:
input_text = f"[[Situation]] {_test_set[33]['title']+' '+_test_set[33]['selftext']}\n"
input_text += f"[[Person X's Comment]] {_test_set[33]['comment']}\n"
input_text += f"[[Person X's Value]] { author_to_subID_to_inst[_author][_test_set[33]['subID']]['abst_value']}\n"
input_text += f"[[Person X's Judgment]] {judgment_to_word[_test_set[33]['judgment']]}\n"
input_text += f"[[Person X's History]]\n"

_fewshot_situations = ""
for idx, _subID in enumerate(_closest_subIDs[:20]):
    _inst = author_to_subID_to_inst[_author][_subID]
    _fewshot_situations += f"\t[Item {idx+1}]\n"
    _fewshot_situations += f"\t\t(Situation) {_inst['title']}\n"
    _fewshot_situations += f"\t\t(Person X's Comment) {_inst['comment']}\n"
    _fewshot_situations += f"\t\t(Person X's Value) {_inst['abst_value']}\n"
    _fewshot_situations += f"\t\t(Person X's Judgment) {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
input_text += _fewshot_situations

prompt_text = FIND_SUPPORT_PROMPT_WITHVALUES.replace("__InputPlaceholder__",input_text)
print(prompt_text)


### Instruction:
You will be given a [Situation] and there's Person X who leaves a [Comment] and provides [Judgment] whether the behavior of original poster of the situation is acceptable or not. You will also be given a list of [Person X's History] of past comments and judgments on similar situations. You need to accomplish two tasks in order. 
{Task 1} Determine which items in the [Person X's History] are mostly RELEVANT to Person X's [Comment] and [Judgment] on a given [Situation]. Note that "RELEVANT ITEMS" should be the supporting evidence you can use to infer Person X's [Comment] and [Judgment] on the [Situation]. You may select up to five items. It is possible that none of the items in the [Person X's History] support their comment and judgment; in this case, the output should be empty.
{Task 2} Using the generated "RELEVANT ITEMS" from {Task 1}, generate short reasoning behind Person X's [Judgment] on the [Situation]. The reasoning should be STRICTLY BASED ON the items selecte

In [12]:
input_text = f"[[Situation]] {_test_set[33]['title']+' '+_test_set[33]['selftext']}\n"
# input_text += f"[[Person X's Comment]] {_test_set[33]['comment']}\n"
input_text += f"[[Person X's Value]] { author_to_subID_to_inst[_author][_test_set[33]['subID']]['abst_value']}\n"
input_text += f"[[Person X's Judgment]] {judgment_to_word[1-_test_set[33]['judgment']]}\n"
input_text += f"[[Person X's History]]\n"

_fewshot_situations = ""
for idx, _subID in enumerate(_closest_subIDs[:20]):
    _inst = author_to_subID_to_inst[_author][_subID]
    _fewshot_situations += f"\t[Item {idx+1}]\n"
    _fewshot_situations += f"\t\t(Situation) {_inst['title']}\n"
    _fewshot_situations += f"\t\t(Person X's Comment) {_inst['comment']}\n"
    _fewshot_situations += f"\t\t(Person X's Value) {_inst['abst_value']}\n"
    _fewshot_situations += f"\t\t(Person X's Judgment) {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
input_text += _fewshot_situations

prompt_text = FIND_FLIPPED_PROMPT_WITHVALUES.replace("__InputPlaceholder__",input_text)
print(prompt_text)


### Instruction:
You will be given a [Situation] and there's Person X who provides [Judgment] whether the behavior of original poster of the situation is acceptable or not. You will also be given a list of [Person X's History] of past comments and judgments on similar situations. You need to accomplish two tasks in order. 
{Task 1} Determine which items in the [Person X's History] are mostly RELEVANT to Person X's [Judgment] on a given [Situation]. Note that "RELEVANT ITEMS" should be the supporting evidence you can use to infer Person X's [Judgment] on the [Situation]. You may select up to five items. It is possible that none of the items in the [Person X's History] support their comment and judgment; in this case, the output should be empty.
{Task 2} Using the generated "RELEVANT ITEMS" from {Task 1}, generate short reasoning behind Person X's [Judgment] on the [Situation]. The reasoning should be STRICTLY BASED ON the items selected from the [Person X's History]. To generate a reas

In [13]:
input_text = f"[[Situation]] {_test_set[33]['title']+' '+_test_set[33]['selftext']}\n"
input_text += f"[[Person X's Comment]] {_test_set[33]['comment']}\n"
input_text += f"[[Person X's Value]] { author_to_subID_to_inst[_author][_test_set[33]['subID']]['abst_value']}\n"
input_text += f"[[Person X's Judgment]] {judgment_to_word[_test_set[33]['judgment']]}\n"
input_text += f"[[Person X's History]]\n"

_fewshot_situations = ""
for idx, _subID in enumerate(_closest_subIDs[-10:]):
    _inst = author_to_subID_to_inst[_author][_subID]
    _fewshot_situations += f"\t[Item {idx+1}]\n"
    _fewshot_situations += f"\t\t(Situation) {_inst['title']}\n"
    _fewshot_situations += f"\t\t(Person X's Comment) {_inst['comment']}\n"
    _fewshot_situations += f"\t\t(Person X's Value) {_inst['abst_value']}\n"
    _fewshot_situations += f"\t\t(Person X's Judgment) {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
input_text += _fewshot_situations

prompt_text = FIND_SUPPORT_PROMPT_WITHVALUES.replace("__InputPlaceholder__",input_text)
print(prompt_text)


### Instruction:
You will be given a [Situation] and there's Person X who leaves a [Comment] and provides [Judgment] whether the behavior of original poster of the situation is acceptable or not. You will also be given a list of [Person X's History] of past comments and judgments on similar situations. You need to accomplish two tasks in order. 
{Task 1} Determine which items in the [Person X's History] are mostly RELEVANT to Person X's [Comment] and [Judgment] on a given [Situation]. Note that "RELEVANT ITEMS" should be the supporting evidence you can use to infer Person X's [Comment] and [Judgment] on the [Situation]. You may select up to five items. It is possible that none of the items in the [Person X's History] support their comment and judgment; in this case, the output should be empty.
{Task 2} Using the generated "RELEVANT ITEMS" from {Task 1}, generate short reasoning behind Person X's [Judgment] on the [Situation]. The reasoning should be STRICTLY BASED ON the items selecte

In [14]:
input_text = f"[[Situation]] {_test_set[33]['title']+' '+_test_set[33]['selftext']}\n"
# input_text += f"[[Person X's Comment]] {_test_set[33]['comment']}\n"
input_text += f"[[Person X's Value]] { author_to_subID_to_inst[_author][_test_set[33]['subID']]['abst_value']}\n"
input_text += f"[[Person X's Judgment]] {judgment_to_word[1-_test_set[33]['judgment']]}\n"
input_text += f"[[Person X's History]]\n"

_fewshot_situations = ""
for idx, _subID in enumerate(_closest_subIDs[-10:]):
    _inst = author_to_subID_to_inst[_author][_subID]
    _fewshot_situations += f"\t[Item {idx+1}]\n"
    _fewshot_situations += f"\t\t(Situation) {_inst['title']}\n"
    _fewshot_situations += f"\t\t(Person X's Comment) {_inst['comment']}\n"
    _fewshot_situations += f"\t\t(Person X's Value) {_inst['abst_value']}\n"
    _fewshot_situations += f"\t\t(Person X's Judgment) {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
input_text += _fewshot_situations

prompt_text = FIND_FLIPPED_PROMPT_WITHVALUES.replace("__InputPlaceholder__",input_text)
print(prompt_text)


### Instruction:
You will be given a [Situation] and there's Person X who provides [Judgment] whether the behavior of original poster of the situation is acceptable or not. You will also be given a list of [Person X's History] of past comments and judgments on similar situations. You need to accomplish two tasks in order. 
{Task 1} Determine which items in the [Person X's History] are mostly RELEVANT to Person X's [Judgment] on a given [Situation]. Note that "RELEVANT ITEMS" should be the supporting evidence you can use to infer Person X's [Judgment] on the [Situation]. You may select up to five items. It is possible that none of the items in the [Person X's History] support their comment and judgment; in this case, the output should be empty.
{Task 2} Using the generated "RELEVANT ITEMS" from {Task 1}, generate short reasoning behind Person X's [Judgment] on the [Situation]. The reasoning should be STRICTLY BASED ON the items selected from the [Person X's History]. To generate a reas

In [22]:
minidf = df[df['author']==_author]
far_subIDs = [elem for elem in minidf.subID.tolist() if elem not in _closest_subIDs]
far_subIDs = random.sample(far_subIDs, 10)

In [23]:
# far_subIDs = [elem for i, elem in enumerate(far_subIDs) if i not in [1,2,3]]
input_text = f"[[Situation]] {_test_set[33]['title']+' '+_test_set[33]['selftext']}\n"
input_text += f"[[Person X's Comment]] {_test_set[33]['comment']}\n"
input_text += f"[[Person X's Value]] { author_to_subID_to_inst[_author][_test_set[33]['subID']]['abst_value']}\n"
input_text += f"[[Person X's Judgment]] {judgment_to_word[_test_set[33]['judgment']]}\n"
input_text += f"[[Person X's History]]\n"

_fewshot_situations = ""
for idx, _subID in enumerate(far_subIDs):
    _inst = author_to_subID_to_inst[_author][_subID]
    _fewshot_situations += f"\t[Item {idx+1}]\n"
    _fewshot_situations += f"\t\t(Situation) {_inst['title']}\n"
    _fewshot_situations += f"\t\t(Person X's Comment) {_inst['comment']}\n"
    _fewshot_situations += f"\t\t(Person X's Value) {_inst['abst_value']}\n"
    _fewshot_situations += f"\t\t(Person X's Judgment) {str(_inst['judgment'])} ({judgment_to_word[_inst['judgment']]})\n"
input_text += _fewshot_situations

prompt_text = FIND_SUPPORT_PROMPT_WITHVALUES.replace("__InputPlaceholder__",input_text)
print(prompt_text)


### Instruction:
You will be given a [Situation] and there's Person X who leaves a [Comment] and provides [Judgment] whether the behavior of original poster of the situation is acceptable or not. You will also be given a list of [Person X's History] of past comments and judgments on similar situations. You need to accomplish two tasks in order. 
{Task 1} Determine which items in the [Person X's History] are mostly RELEVANT to Person X's [Comment] and [Judgment] on a given [Situation]. Note that "RELEVANT ITEMS" should be the supporting evidence you can use to infer Person X's [Comment] and [Judgment] on the [Situation]. You may select up to five items. It is possible that none of the items in the [Person X's History] support their comment and judgment; in this case, the output should be empty.
{Task 2} Using the generated "RELEVANT ITEMS" from {Task 1}, generate short reasoning behind Person X's [Judgment] on the [Situation]. The reasoning should be STRICTLY BASED ON the items selecte